Low amount of data and high number of features, so we need a classification model that has bias/low variance such as Linear SVM.

https://www.kdnuggets.com/2020/05/guide-choose-right-machine-learning-algorithm.html

In [1]:
import os
os.chdir('..')
os.chdir('../resources')
pwd = os.getcwd()

In [2]:
%%time
import pandas as pd
import cv2

features = []
pixels = []
img_names = []
feature_names = ['head', 'spine']#, 'leg', 'arm', 'hand', 'foot']
for feature in feature_names+ ['none']:
    file_names = os.listdir(pwd+'/features/'+feature)
    img_names = img_names + file_names
    for file in file_names:
        features.append(feature)
        pixels.append(cv2.imread(pwd+'/features/'+feature+'/'+file, 0))

feature_images = pd.DataFrame({'image_name': img_names,'class' : features, 'image_pixels': pixels})

print('Total features: ', feature_images.shape[0])
print(feature_images.value_counts('class'))
feature_images

Total features:  586
class
head     357
none     167
spine     62
dtype: int64
Wall time: 792 ms


,image_name,class,image_pixels
0,19150_2021-01-24_10.31_103052326.png,head,"[[79, 78, 82, 85, 87, 89, 93, 97, 100, 104, 10..."
1,19158_2021-01-09_19.37_193140441.png,head,"[[77, 83, 88, 93, 98, 102, 104, 104, 103, 105,..."
2,19158_2021-01-10_19.16_19114035.png,head,"[[133, 132, 129, 122, 115, 114, 113, 116, 129,..."
3,19162_2021-01-08_13.46_13434478.png,head,"[[119, 124, 121, 114, 108, 102, 99, 100, 106, ..."
4,19162_2021-01-08_14.14_141028517.png,head,"[[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, ..."
...,...,...,...
581,19642_2021-01-16_18.54_185155873.png,none,"[[120, 122, 139, 167, 182, 182, 177, 177, 186,..."
582,19642_2021-01-17_10.15_10101114.png,none,"[[171, 161, 165, 155, 158, 175, 177, 167, 171,..."
583,19642_2021-01-17_10.15_101520226.png,none,"[[142, 174, 197, 203, 198, 207, 215, 215, 171,..."
584,19676_2021-02-04_21.33_21322923.png,none,"[[68, 66, 70, 76, 77, 74, 73, 73, 69, 70, 74, ..."


Get images ready for HOG:
1. Filter
2. Resize to (128, 128)

In [3]:
%%time
from Process import process
def resize(x):
    return cv2.resize(x,(128, 128))

#feature_images['filtered_pixels'] = feature_images['image_pixels'].apply(process)
feature_images['resized_pixels'] = feature_images['image_pixels'].apply(resize)
feature_images

Wall time: 113 ms


,image_name,class,image_pixels,resized_pixels
0,19150_2021-01-24_10.31_103052326.png,head,"[[79, 78, 82, 85, 87, 89, 93, 97, 100, 104, 10...","[[79, 79, 82, 85, 87, 91, 95, 99, 103, 106, 10..."
1,19158_2021-01-09_19.37_193140441.png,head,"[[77, 83, 88, 93, 98, 102, 104, 104, 103, 105,...","[[77, 82, 87, 92, 96, 100, 103, 104, 103, 104,..."
2,19158_2021-01-10_19.16_19114035.png,head,"[[133, 132, 129, 122, 115, 114, 113, 116, 129,...","[[133, 132, 126, 118, 115, 114, 119, 130, 132,..."
3,19162_2021-01-08_13.46_13434478.png,head,"[[119, 124, 121, 114, 108, 102, 99, 100, 106, ...","[[119, 123, 121, 116, 110, 105, 101, 99, 101, ..."
4,19162_2021-01-08_14.14_141028517.png,head,"[[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, ...","[[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, ..."
...,...,...,...,...
581,19642_2021-01-16_18.54_185155873.png,none,"[[120, 122, 139, 167, 182, 182, 177, 177, 186,...","[[121, 127, 149, 174, 182, 179, 177, 186, 198,..."
582,19642_2021-01-17_10.15_10101114.png,none,"[[171, 161, 165, 155, 158, 175, 177, 167, 171,...","[[171, 161, 165, 155, 158, 173, 177, 168, 170,..."
583,19642_2021-01-17_10.15_101520226.png,none,"[[142, 174, 197, 203, 198, 207, 215, 215, 171,...","[[142, 175, 195, 196, 206, 215, 191, 153, 140,..."
584,19676_2021-02-04_21.33_21322923.png,none,"[[68, 66, 70, 76, 77, 74, 73, 73, 69, 70, 74, ...","[[69, 67, 72, 76, 74, 73, 72, 69, 71, 72, 63, ..."


Compute HOG features

In [4]:
%%time
from Hog import compute_hog
feature_images['hog'] = feature_images['resized_pixels'].apply(compute_hog)
feature_images

Wall time: 2.1 s


,image_name,class,image_pixels,resized_pixels,hog
0,19150_2021-01-24_10.31_103052326.png,head,"[[79, 78, 82, 85, 87, 89, 93, 97, 100, 104, 10...","[[79, 79, 82, 85, 87, 91, 95, 99, 103, 106, 10...","[[0.04839468071532554, 0.02948810026201215, 0...."
1,19158_2021-01-09_19.37_193140441.png,head,"[[77, 83, 88, 93, 98, 102, 104, 104, 103, 105,...","[[77, 82, 87, 92, 96, 100, 103, 104, 103, 104,...","[[0.02191214639811905, 0.02670508651597998, 0...."
2,19158_2021-01-10_19.16_19114035.png,head,"[[133, 132, 129, 122, 115, 114, 113, 116, 129,...","[[133, 132, 126, 118, 115, 114, 119, 130, 132,...","[[0.04519320454669157, 0.04018420527470465, 0...."
3,19162_2021-01-08_13.46_13434478.png,head,"[[119, 124, 121, 114, 108, 102, 99, 100, 106, ...","[[119, 123, 121, 116, 110, 105, 101, 99, 101, ...","[[0.06021372343813004, 0.035893074411894065, 0..."
4,19162_2021-01-08_14.14_141028517.png,head,"[[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, ...","[[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...
581,19642_2021-01-16_18.54_185155873.png,none,"[[120, 122, 139, 167, 182, 182, 177, 177, 186,...","[[121, 127, 149, 174, 182, 179, 177, 186, 198,...","[[0.04730741724291017, 0.02954489923818306, 0...."
582,19642_2021-01-17_10.15_10101114.png,none,"[[171, 161, 165, 155, 158, 175, 177, 167, 171,...","[[171, 161, 165, 155, 158, 173, 177, 168, 170,...","[[0.06891262877289671, 0.06664489370117056, 0...."
583,19642_2021-01-17_10.15_101520226.png,none,"[[142, 174, 197, 203, 198, 207, 215, 215, 171,...","[[142, 175, 195, 196, 206, 215, 191, 153, 140,...","[[0.11648649506187697, 0.06878848092414896, 0...."
584,19676_2021-02-04_21.33_21322923.png,none,"[[68, 66, 70, 76, 77, 74, 73, 73, 69, 70, 74, ...","[[69, 67, 72, 76, 74, 73, 72, 69, 71, 72, 63, ...","[[0.022663235076007475, 0.029759945757108445, ..."


Separate into test and train data

Train Linear SVMs for each feature

In [34]:
%%time
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
feature_svm = []
fully_trained = []
accuracy = []
classification_rep = []
for feature in feature_names:
    df = feature_images[feature_images['class'].isin([feature, 'none'])]
    
    labels =  np.array(df['class']).reshape(len(df['class']),1)
    hog_features = np.stack(df['hog'], axis=1)[0]
    data = np.hstack((hog_features,labels))

    np.random.shuffle(data)
    percentage = 80
    partition = int(len(data)*percentage/100)

    x_train, x_test = data[:partition,:-1],  data[partition:,:-1]
    y_train, y_test = data[:partition,-1:].ravel() , data[partition:,-1:].ravel()

    clf = svm.SVC(probability=True)
    clf.fit(x_train,y_train)
    
    clf2 = svm.SVC(probability=True)
    clf2.fit(data[:len(data),:-1],data[:len(data),-1:].ravel())
    
    y_pred = clf.predict(x_test)
    
    fully_trained.append(clf2)
    feature_svm.append(clf)
    accuracy.append(accuracy_score(y_test, y_pred))
    classification_rep.append(classification_report(y_test, y_pred))

results = pd.DataFrame({'feature': feature_names, 'svm': feature_svm, 'accuracy': accuracy, 'classification report': classification_rep})

Wall time: 1.53 s


Print results

In [35]:
for idx,row in results.iterrows():
    print(row['feature'])
    print('accuracy: '+str(row['accuracy']))
    print(row['classification report'])
    print('\n')

head
accuracy: 0.9238095238095239
              precision    recall  f1-score   support

        head       0.93      0.96      0.95        73
        none       0.90      0.84      0.87        32

    accuracy                           0.92       105
   macro avg       0.92      0.90      0.91       105
weighted avg       0.92      0.92      0.92       105



spine
accuracy: 0.8478260869565217
              precision    recall  f1-score   support

        none       0.89      0.92      0.90        36
       spine       0.67      0.60      0.63        10

    accuracy                           0.85        46
   macro avg       0.78      0.76      0.77        46
weighted avg       0.84      0.85      0.84        46





In [37]:
import pickle
from pathlib import Path

svm_folder =  pwd+"/SVM/"
for clf in fully_trained:
    class_ = clf.classes_
    class_ = np.copy(class_).tolist()
    class_.remove('none')
    pkl_filename = class_[0]+'.pkl'
    with open(svm_folder+pkl_filename, 'wb+') as file:
        pickle.dump(clf, file)

In [ ]:
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)